In [1]:
import pyscf
import pyscf.tools

In [2]:
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 Fe -1.67785607 -0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
"""

In [3]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5351.41478186842
HOMO (B) = 0.318236980196628  LUMO (A) = 0.320979897364191
cycle= 1 E= -5347.01305352181  delta_E=  4.4  |g|= 4.93  |ddm|= 11.8
HOMO (B) = -0.208577216881658  LUMO (A) = 0.00879930358550486
cycle= 2 E= -5326.86878641886  delta_E= 20.1  |g|= 8.88  |ddm|= 13.1
HOMO (B) = 1.25634475236875  LUMO (B) = 0.0701291625722303
cycle= 3 E= -5328.84109730651  delta_E= -1.97  |g|= 5.69  |ddm|= 29.9
HOMO (A) = 0.189978800807618  LUMO (B) = 0.32296698202972
cycle= 4 E= -5355.41569900171  delta_E= -26.6  |g|= 0.

In [4]:
F = mf.get_fock()

In [77]:
import numpy as np
import scipy
import copy as cp
import math

def get_frag_bath(Pin, frag, S, thresh=1e-7, verbose=2):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Xinv = scipy.linalg.inv(X)

    Nbas = S.shape[0]
    Cfrag = Xinv@np.eye(Nbas)[:,frag]
    

    P = X@Pin@X.T

    nfrag = np.trace(P[frag,:][:,frag])
    P[frag,:] = 0
    P[:,frag] = 0
    bath_idx = []
    env_idx = []
    e,U = np.linalg.eigh(P)
    nbath = 0.0
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            env_idx.append(nidx)
        elif thresh < ni < 1-thresh:
            if verbose > 1:
                print(" eigvalue: %12.8f" % ni)
            bath_idx.append(nidx)
            nbath += ni
        
            
    print(" # Electrons frag: %12.8f bath: %12.8f total: %12.8f" %(nfrag, nbath, nfrag+nbath))
    Cenv = Xinv@U[:,env_idx]
    Cbath = Xinv@U[:,bath_idx]
    
    
    C = np.hstack((Cfrag, Cbath))
    
    # Get virtual orbitals (these are just the orthogonal complement of the env and frag/bath
    Q = np.eye(C.shape[0]) - X@C@C.T@X - X@Cenv@Cenv.T@X
    e,U = np.linalg.eigh(Q)
    vir_idx = []
    for nidx,ni in enumerate(e):
        if math.isclose(ni, 1, abs_tol=thresh):
            vir_idx.append(nidx)
    Cvir = Xinv@U[:,vir_idx]

    assert(Cenv.shape[1] + Cvir.shape[1] + C.shape[1] == Nbas)

          
    # print(C.T@S@C)
    return (Cenv, C, Cvir)

def gram_schmidt(frags, S, thresh=1e-8):
    # |v'> = (1-sum_i |i><i|) |v>
    #      = |v> - sum_i |i><i|v>
    Nbas = S.shape[1]
    seen = []
    out = []
    seen = np.zeros((Nbas, 0))

    for f in frags:
        outf = np.zeros((Nbas, 0))
        # grab each orbital
        for fi in range(f.shape[1]):
            v = f[:,fi]
            v.shape = (Nbas, 1)

            # Compare to previous orbitals
            for fj in range(seen.shape[1]):
                j = seen[:,fj]
                j.shape = (Nbas, 1)
                ovlp = (j.T @ S @ v)[0]
                v = v - j * ovlp

                norm = np.sqrt((v.T @ S @ v)[0])
                if norm < thresh:
                    print(" Warning: small norm in GS: ", norm)
                v = v/norm
            
            outf = np.hstack((outf, v))
            seen = np.hstack((seen, v))
        out.append(outf)
    return out

def sym_ortho(frags, S, thresh=1e-8):
    Nbas = S.shape[1]
    
    inds = []
    Cnonorth = np.hstack(frags)
    shift = 0
    for f in frags:
        inds.append(list(range(shift, shift+f.shape[1])))
        shift += f.shape[1]
        
    
    Smo = Cnonorth.T @ S @ Cnonorth
    X = np.linalg.inv(scipy.linalg.sqrtm(Smo))
    # print(Cnonorth.shape, X.shape)
    Corth = Cnonorth @ X
    
    frags2 = []
    for f in inds:
        frags2.append(Corth[:,f])
    return frags2

def get_spade_orbitals(orb_list, C, S, thresh=1e-6):
    """
    Find the columns in C that span the rows specified by orb_list
    """
    
    # First orthogonalize C
    X = scipy.linalg.sqrtm(S)
    Corth = X @ C
    
    U,s,V = np.linalg.svd(Corth[orb_list,:])
    nkeep = 0
    for idx,si in enumerate(s):
        if si > thresh:
            nkeep += 1
        print(" Singular value: ", si)
    print(" # of orbitals to keep: ", nkeep)
    
    Xinv = scipy.linalg.inv(X)
    
    Csys = Xinv @ Corth @ V[0:nkeep,:].T
    Cenv = Xinv @ Corth @ V[nkeep::,:].T
    return Csys, Cenv

def semi_canonicalize(C,F):
    e,V = np.linalg.eigh(C.T @ F @ C)
    for ei in e:
        print(" Orbital Energy: ", ei)
    return C @ V

In [78]:
import scipy

# collect 2s,2p from O in Occ add to singly occupied orbitals
docc_list = []
socc_list = []
virt_list = []
oxygen_list = []
for idx,i in enumerate(mf.mo_occ):
    if i == 0:
        virt_list.append(idx)
    elif i == 1:
        socc_list.append(idx)
    elif i == 2:
        docc_list.append(idx)
        
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 2:
        if ao[2] in ("2s", "2p"):
            oxygen_list.append(ao_idx)

C = mf.mo_coeff
S = mf.get_ovlp()
P = mf.make_rdm1()
Pa = P[0,:,:]
Pb = P[1,:,:]

Cdocc = C[:,docc_list]
Csocc = C[:,socc_list]
Cvirt = C[:,virt_list]

Cfrag, Cenv = get_spade_orbitals(oxygen_list, Cdocc, S)
pyscf.tools.molden.from_mo(mf.mol, "Cfrag.molden", Cfrag);



 Singular value:  0.6897009857222781
 Singular value:  0.6697854732763284
 Singular value:  0.6475976877712083
 Singular value:  0.6462488919632164
 # of orbitals to keep:  4


# Split into Clusters

In [88]:
frag1 = []
frag2 = []
frag3 = []

for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        frag1.append(ao_idx)
    elif ao[0] == 1:
        frag2.append(ao_idx)    
    elif ao[0] == 2:
        frag3.append(ao_idx)


C1, C1env = get_spade_orbitals(frag1, Cact, S, thresh=.5)
C2, C2env = get_spade_orbitals(frag2, Cact, S, thresh=.5)
C3, C2env = get_spade_orbitals(frag3, Cact, S, thresh=.5)
C1, C2, C3 = sym_ortho([C1, C2, C3], S)

C1 = semi_canonicalize(C1, F)
C2 = semi_canonicalize(C2, F)
C3 = semi_canonicalize(C3, F)

Cact = np.hstack((C1,C2, C3))
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact);


 Singular value:  0.9918742904745268
 Singular value:  0.989594642091932
 Singular value:  0.9848888178307931
 Singular value:  0.976335594913786
 Singular value:  0.9618203274591715
 Singular value:  0.4124937447852499
 Singular value:  0.1937350334744497
 Singular value:  0.18368352200397497
 Singular value:  0.05298722871780828
 Singular value:  0.011125439221798324
 Singular value:  0.002463880096175464
 Singular value:  0.0011885312332925908
 Singular value:  0.0008209187990010771
 Singular value:  0.00019677604070295814
 Singular value:  4.510074808879434e-09
 Singular value:  2.356734613396517e-09
 Singular value:  8.425015524805122e-10
 Singular value:  3.9425224811698496e-10
 # of orbitals to keep:  5
 Singular value:  0.9918742905371501
 Singular value:  0.9895946420941244
 Singular value:  0.9848888177384579
 Singular value:  0.9763355949826049
 Singular value:  0.9618203273775107
 Singular value:  0.41249374507402337
 Singular value:  0.19373503260919414
 Singular value:  0

# Check that we still have all our electrons

In [89]:
Cact = np.hstack((Cfrag, Csocc))
na_act = np.trace(Cact.T @ S @ Pa @ S @ Cact)
na_env = np.trace(Cenv.T @ S @ Pa @ S @ Cenv)
na_vir = np.trace(Cvirt.T @ S @ Pa @ S @ Cvirt)
nb_act = np.trace(Cact.T @ S @ Pb @ S @ Cact)
nb_env = np.trace(Cenv.T @ S @ Pb @ S @ Cenv)
nb_vir = np.trace(Cvirt.T @ S @ Pb @ S @ Cvirt)
print(" # electrons: %12s %12s" %("α", "β"))
print("         Env: %12.8f %12.8f" %(na_env, nb_env))
print("         Act: %12.8f %12.8f" %(na_act, nb_act))
print("         Vir: %12.8f %12.8f" %(na_vir, nb_vir))

 # electrons:            α            β
         Env:  73.00000000  73.00000000
         Act:  14.00000000   4.00000000
         Vir:   0.00000000   0.00000000


# Make Integrals

In [90]:
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

In [91]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [92]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [93]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [94]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)